In [1]:
# import necessary packages
!pip install -r requirements.txt/
!pip install wbdata

import ndb
import pandas as pd
import warnings
from  scipy.optimize import linprog as lp
import numpy as np
import wbdata 

Could not open requirements file: [Errno 20] Not a directory: 'requirements.txt/'


In [2]:
SHEETs = [# All Food
          ("https://docs.google.com/spreadsheet/ccc?key=1nkwKO01nDmLCHqiq4cE1N1LQYM-oNgrvhkwB92PCtE4","All"), 
         ]

In [3]:
import ndb
import pandas as pd
import warnings

DFs = []
#for csv in CSVs:  # Uncomment to use a list of csv files as inputs
#  DFs.append(pd.read_csv(csv,dtype=str))

try:
    if len(SHEETs):
        for ID, RANGE_NAME in SHEETs:
            try:
                if "docs.google.com" in ID:
                    sheet = "%s&output=csv" % ID
                else:
                    sheet = "https://docs.google.com/spreadsheet/ccc?key=%s&output=csv" % ID
                DFs.append(pd.read_csv(sheet))
            except ParserError:
                warnings.warn("Can't read sheet at https://docs.google.com/spreadsheets/d/%s.\nCheck Sharing settings, so that anyone with link can view?" % ID)
except NameError: # SHEETS not defined?
    pass

df = pd.concat(DFs,ignore_index=True,sort=False)
# Some columns which ought to be numeric are actually str; convert them
df['Price'] = df['Price'].astype(float)
df['Quantity'] = df['Quantity'].astype(float)
df["Units"] = df["Units"].astype(str)
df["Value"] = df["Value"].dropna().astype(int).astype(str)
df["NDB"][df['Food'].str.contains("Milk shakes and other dairy drinks")] = str("01110")
df["NDB"] = df["NDB"].dropna()

df

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Value,Food,Quantity,Units,Price,Date,Location,NDB,Notes,Unnamed: 9
0,1002,"Milk, whole",1.00,gallon,4.89,03/19/2019,Safeway - 94704,45287045,0.5 gallons,94704 zipcode
1,1004,"Milk, reduced fat",1.00,gallon,6.88,03/19/2019,Safeway,45243941,0.5 gallons,NaN
2,1006,"Milk, low fat",1.00,gallon,6.88,03/19/2019,Safeway,45243941,0.5 gallons,NaN
3,1008,"Milk, non fat",1.00,gallon,3.39,03/19/2019,Safeway,45340014,0.5 gallons,NaN
4,1202,"Flavored milk,whole",0.50,gallon,3.39,03/19/2019,Safeway,14177,Chocolate milk,NaN
5,1204,"Flavored milk, reduced fat",12.00,oz,1.99,03/19/2019,Safeway,45236702,NaN,NaN
6,1206,"Flavored milk, low fat",0.50,gallon,4.59,03/19/2019,Safeway,45289703,Chocolate milk,NaN
7,1402,Milk shakes and other dairy drinks,14.00,oz,3.00,03/19/2019,Safeway,01110,NaN,NaN
8,1404,Milk substitutes,64.00,oz,5.69,03/19/2019,Safeway,45325820,Organic coconut milk,NaN
9,1602,Cheese,32.00,oz,6.99,03/19/2019,Safeway,45102597,1 lb,NaN


In [4]:
values = df["Value"].tolist()
names = df["Food"].tolist()
value_name_dict = dict(zip(values,names))

In [5]:
user = "atwater"
apikey = {'atwater':"ekh9KVmwQxokFmje0yDKJsSdG5PCuxWpXphJMEB3"}
age_list = ["C 1-3", "F 4-8", "M 4-8", "F 9-13", "M 9-13", "F 14-18", "M 14-18", "F 19-30", "M 19-30",\
            "F 31-50", "M 31-50", "F 51+", "M 51+"]
#http://www.censusscope.org/us/chart_age.html
age_breakdown = [6.82, 3.56, 3.74, 3.56, 3.74, 3.49, 3.69, 6.71, 6.92,\
                15.32, 15.15, 15.01, 12.32]
    
    
import ndb
D = {}

for food in  df.Food.tolist():
    try:
        NDB = df.loc[df.Food==food,:].NDB
        D[food] = ndb.ndb_report(apikey[user],NDB).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find NDB Code %s for food %s." % (food,NDB))
        

D = pd.DataFrame(D,dtype=float)

D

/home/jovyan/Atwater-Min-Cost-Diet/ndb.py:40: UserWarning: Couldn't find NDB=143    NaN
Name: NDB, dtype: object.
  warnings.warn("Couldn't find NDB=%s." % ndbno)
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Couldn't find NDB Code Not included in a food category for food 143    NaN
Name: NDB, dtype: object.
/home/jovyan/Atwater-Min-Cost-Diet/ndb.py:40: UserWarning: Couldn't find NDB=144    NaN
Name: NDB, dtype: object.
  warnings.warn("Couldn't find NDB=%s." % ndbno)
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Couldn't find NDB Code Item not assigned a food code for food 144    NaN
Name: NDB, dtype: object.


,"Milk, whole","Milk, reduced fat","Milk, low fat","Milk, non fat","Flavored milk,whole","Flavored milk, reduced fat","Flavored milk, low fat",Milk shakes and other dairy drinks,Milk substitutes,Cheese,...,"Dips, gravies, other sauces",Sugars and honey,Sugar substitutes,"Jams, syrups, toppings",Baby food: cereals,Baby food: fruit,Baby food: vegetable,Baby food: meat and dinners,Baby food: snacks and sweets,"Formula, ready-to-feed"
Caffeine,NaN,NaN,NaN,NaN,3.000,NaN,NaN,2.000,NaN,NaN,...,0.000,0.000,NaN,NaN,0.000,0.000,0.000,0.000,0.000,0.000
"Calcium, Ca",125.00,125.000,125.000,139.00,95.000,125.00,125.00,132.000,0.00,714.00,...,26.000,83.000,NaN,53.00,860.000,10.000,19.000,5.000,10.000,69.000
"Carbohydrate, by difference",5.00,5.000,5.000,5.83,11.960,17.92,10.83,21.150,6.67,7.14,...,15.890,98.090,100.0,70.00,83.140,13.600,5.230,2.430,13.600,6.110
Cholesterol,15.00,8.000,8.000,2.00,9.000,8.00,4.00,11.000,0.00,107.00,...,0.000,0.000,NaN,0.00,0.000,0.000,0.000,51.000,0.000,0.000
Energy,67.00,54.000,54.000,38.00,85.000,104.00,67.00,119.000,550.00,357.00,...,119.000,380.000,0.0,500.00,390.000,57.000,21.000,81.000,57.000,63.000
"Fatty acids, total monounsaturated",NaN,0.620,0.620,NaN,0.829,NaN,NaN,0.780,NaN,NaN,...,0.859,0.000,NaN,NaN,0.420,0.043,0.012,0.950,0.043,1.292
"Fatty acids, total polyunsaturated",NaN,0.000,0.000,NaN,0.186,NaN,NaN,0.100,NaN,NaN,...,1.957,0.000,NaN,NaN,0.784,0.046,0.148,0.160,0.046,0.651
"Fatty acids, total saturated",2.08,1.250,1.250,0.00,1.860,1.25,0.62,1.681,40.00,17.86,...,0.529,0.000,NaN,13.33,0.234,0.010,0.052,1.180,0.010,1.443
"Fatty acids, total trans",0.00,0.000,0.000,0.00,NaN,0.00,0.00,NaN,0.00,NaN,...,0.008,NaN,NaN,0.00,0.000,NaN,NaN,NaN,NaN,NaN
"Fiber, total dietary",0.00,0.000,0.000,0.00,0.400,0.00,0.00,0.300,0.00,0.00,...,4.900,0.000,NaN,3.30,1.300,1.200,2.300,0.000,1.200,0.000


In [6]:
# Convert food quantities to NDB units
df['NDB Quantity'] = df[['Quantity','Units']].T.apply(lambda x : ndb.ndb_units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['NDB Price'] = df['Price']/df['NDB Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food')['NDB Price'].min()

Prices.head()

Food
Apple juice                    0.11023113109243877 / hectogram
Apples                          0.3012984249859993 / hectogram
Baby food: cereals              1.5388265900504454 / hectogram
Baby food: fruit                1.0097171608067392 / hectogram
Baby food: meat and dinners     2.1023281321949923 / hectogram
Name: NDB Price, dtype: object

In [7]:
# Choose sex/age group:
group = "M 19-30"

# Define *minimums*
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

# Define *maximums*
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]

In [8]:

from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

c = Prices.apply(lambda x:x.magnitude).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = list(set(c.index.tolist()).intersection(D.columns.tolist()))
c = c[use]

# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[c.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([-Amin,Amax])

b = pd.concat([-bmin,bmax]) # Note sign change for min constraints

# Now solve problem!
result = lp(c, A, b, method='interior-point')

# Put back into nice series
diet = pd.Series(result.x,index=c.index)

print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))
print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.

tab = pd.DataFrame({"Outcome":np.abs(A).dot(diet),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())

Cost of diet for M 19-30 is $1.33 per day.

You'll be eating (in 100s of grams or milliliters):
Food
Nutritional beverages    1.200000
Bananas                  4.540235
dtype: float64

With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000000          2400.0
Protein                           86.985176            56.0
Fiber, total dietary              45.402353            33.6
Folate, DFE                     1165.200000           400.0
Calcium, Ca                     1033.869177          1000.0
Carbohydrate, by difference      330.921882           130.0
Iron, Fe                          53.106353             8.0
Magnesium, Mg                    600.000000           400.0
Niacin                            30.000000            16.0
Phosphorus, P                   1714.800000           700.0
Potassium, K                    5081.90117

In [9]:
all_households = pd.read_csv('faps_household_puf.csv', dtype=str)

all_households.head()


,hhnum,initintrvmon,startmon,initfinaldays,initialdate_flag,startdate_edit,startlag,matchconsenthh,nonmetro,region,...,feedback2,feedback3,feedback4_1,feedback4_2,feedback4_3,feedback4_4,feedback4_5,feedback4_6,feedback4_7,feedback4_8
0,100012,1,1,8,0,0,1,1,1,3,...,3,3,0,0,0,0,0,0,0,1
1,100015,8,8,8,0,0,1,1,0,3,...,6,2,0,0,0,0,0,0,0,1
2,100024,6,6,9,0,0,0,1,0,2,...,1,1,0,0,0,0,0,0,0,1
3,100026,7,7,10,0,0,1,1,0,3,...,1,1,0,0,0,0,0,0,0,1
4,100028,5,5,8,0,0,1,1,0,1,...,4,1,0,0,0,0,0,0,0,1


In [10]:
northeast = all_households[all_households["region"] == "1"]
midwest = all_households[all_households["region"] == "2"]
south = all_households[all_households["region"] == "3"]
west = all_households[all_households["region"] == "4"]

In [11]:
hh_choice = pd.read_csv('faps_fafhnutrient_puf.csv', dtype=str)

hh_choice.head(30)

,hhnum,eventid,itemnum,foodcode,foodcodetype,codenot1112,usdadescmain,usdadescadd,usdafoodcat1,usdafoodcat2,...,mfa181,mfa201,mfa221,pfa182,pfa183,pfa184,pfa204,pfa205,pfa225,pfa226
0,103698,5895,1,24198700,1,0,"Chicken patty, fillet, or tenders, breaded, co...",Weaver Chicken Breast Tenders; Tyson Breaded C...,2,22,...,3.852,0.059,0.002,5.897,0.196,0.001,0.024,0,0.003,0.002
1,103698,5895,2,75141000,1,0,"Cabbage salad or coleslaw, made with coleslaw ...",NaN,6,64,...,2.046,0,0,3.636,0.452,0,0,0,0,0
2,103698,5895,3,52209010,1,0,Hush puppy,fried cornbread,4,44,...,7.73,0.054,0.002,1.626,0.116,0.001,0.004,0.009,0,0
3,103698,5895,4,92400100,1,0,"Soft drink, NFS, sugar-free",diet,7,71,...,0,0,0,0,0,0,0,0,0,0
4,107440,5896,1,71401030,1,0,"White potato, french fries, from frozen, deep ...",NaN,6,68,...,5.821,0.107,0.004,4.948,0.436,0,0.004,0,0,0
5,107440,5896,2,27510311,1,0,"Cheeseburger, 1 oz meat, plain, on miniature bun",NaN,3,37,...,4.424,0.046,0,1.077,0.12,0.001,0.007,0.003,0.009,0.001
6,107440,5896,3,92410320,1,0,"Soft drink, cola-type, sugar-free",Diet Coke; Diet Pepsi; Tab; Coca Cola Zero; Pe...,7,71,...,0,0,0,0,0,0,0,0,0,0
7,107440,5896,4,92410550,1,0,"Soft drink, fruit flavored, caffeine containing",Mellow Yellow; Mountain Dew; Big Red; 7-Up Gol...,7,72,...,0,0,0,0,0,0,0,0,0,0
8,107440,5896,5,27510600,1,0,"Hamburger, 1 oz meat, plain, on miniature bun",NaN,3,37,...,3.949,0.016,0,1.089,0.117,0,0,0.001,0.006,0
9,103790,5905,1,92650000,1,1,Red Bull Energy Drink,NaN,7,72,...,0,0,0,0,0,0,0,0,0,0


In [12]:
northeast_hh = hh_choice[hh_choice["hhnum"].isin(northeast["hhnum"])]
midwest_hh = hh_choice[hh_choice["hhnum"].isin(midwest["hhnum"])]
south_hh = hh_choice[hh_choice["hhnum"].isin(south["hhnum"])]
west_hh = hh_choice[hh_choice["hhnum"].isin(west["hhnum"])]

northeast_hh['gramstotal'] = northeast_hh['gramstotal'].astype(float)
midwest_hh['gramstotal'] = midwest_hh['gramstotal'].astype(float)
south_hh['gramstotal'] = south_hh['gramstotal'].astype(float)
west_hh['gramstotal'] = west_hh['gramstotal'].astype(float)

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [13]:
hh_choice["food"] = hh_choice["usdafoodcat4"].map(value_name_dict)

In [14]:
hh_choice[["hhnum","usdafoodcat4","food","gramstotal"]]

,hhnum,usdafoodcat4,food,gramstotal
0,103698,2204,"Chicken patties, nuggets and tenders",52
1,103698,6422,Vegetable mixed dishes,113
2,103698,4402,"Biscuits, muffins, quick breads",42
3,103698,7102,Diet soft drinks,960
4,107440,6804,French fries and other fried potatoes,157
5,107440,3702,Burgers,226
6,107440,7102,Diet soft drinks,622
7,107440,7202,Soft drinks,647
8,107440,3702,Burgers,55
9,103790,7206,Sports and energy drinks,480


In [15]:
hh_choice['gramstotal'] = hh_choice['gramstotal'].astype(float)

In [16]:
hh_choice.groupby(["hhnum", "food"])["gramstotal"].sum().unstack('food')

food,Apple juice,Apples,Baby food: cereals,Baby food: fruit,Baby food: meat and dinners,Baby food: snacks and sweets,Baby food: vegetable,Bacon,Bagels and English muffins,Bananas,...,Tortillas,"Turkey, duck, other poultry",Turnovers and other grain-based items,Vegetable juice,Vegetable mixed dishes,"White potatoes, baked or boiled",Wine,Yeast breads,"Yogurt, low-fat and nonfat","Yogurt, whole and reduced fat"
hhnum,,,,,,,,,,,,,,,,,,,,,
100012,1489.36227,113.889794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,109.936444,NaN,NaN,NaN,NaN
100015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,544.950943,NaN,NaN
100024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,348.0,NaN,NaN,NaN
100026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,162.889894,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100028,135.39657,113.889794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,990.000000,NaN,NaN,NaN,NaN,NaN
100031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.548351,NaN,NaN,...,NaN,NaN,NaN,NaN,816.480000,NaN,NaN,NaN,NaN,NaN
100049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100053,NaN,NaN,NaN,NaN,NaN,83.45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,464.000000,NaN,NaN,52.900000,NaN,NaN


In [17]:
hh_choice.groupby("food")["gramstotal"].sum() / len(hh_choice['hhnum'].unique())

food
Apple juice                                40.842922
Apples                                     40.109108
Baby food: cereals                          0.197561
Baby food: fruit                            0.401690
Baby food: meat and dinners                 0.054050
Baby food: snacks and sweets                0.207198
Baby food: vegetable                        0.171190
Bacon                                       5.023131
Bagels and English muffins                 11.778680
Bananas                                    22.571184
Beans, peas, legumes                       40.562574
Beef, excludes ground                      40.639739
Beer                                      156.000767
Berries                                     7.661380
Biscuits, muffins, quick breads            32.001046
Bottled water                             221.797966
Burgers                                   241.522388
Burritos and tacos                        227.417230
Butter and animal fats                   